# Project-27

# Introduction

# Preliminary Results


## Required Packages

In [1]:
library(tidyverse)
library(base)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Reading data from the web

In [2]:
hastings_2023 <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/crimedata_csv_Hastings-Sunrise_2023.csv")
hastings_2022 <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/crimedata_csv_Hastings-Sunrise_2022.csv")
west_point_grey_2022 <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/crimedata_csv_West%20Point%20Grey_2022.csv")
west_point_grey_2023 <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/crimedata_csv_West%20Point%20Grey_2023.csv")

Rows: 482 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 946 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 238 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this

In [3]:
head(hastings_2023)
head(hastings_2022)
head(west_point_grey_2022)
head(west_point_grey_2023)

TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2023,3,2,16,0,11XX NOOTKA ST,Hastings-Sunrise,496992.1,5457928
Theft from Vehicle,2023,4,13,10,59,12XX KAMLOOPS ST,Hastings-Sunrise,496017.8,5457847
Theft from Vehicle,2023,2,28,0,0,12XX NOOTKA ST,Hastings-Sunrise,496992.2,5457899
Theft from Vehicle,2023,2,3,2,20,14XX RENFREW ST,Hastings-Sunrise,496790.2,5457683
Theft from Vehicle,2023,1,30,2,11,16XX LILLOOET ST,Hastings-Sunrise,497162.8,5457456
Theft from Vehicle,2023,4,13,22,30,16XX NANAIMO ST,Hastings-Sunrise,495887.3,5457516


TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2022,5,16,4,20,11XX NOOTKA ST,Hastings-Sunrise,496992.1,5457928
Theft from Vehicle,2022,10,9,2,0,11XX ROSSLAND ST,Hastings-Sunrise,496691.3,5457991
Theft from Vehicle,2022,4,28,17,0,12XX LILLOOET ST,Hastings-Sunrise,497174.1,5457852
Theft from Vehicle,2022,4,17,5,17,12XX ROSSLAND ST,Hastings-Sunrise,496689.9,5457876
Theft from Vehicle,2022,11,8,22,0,12XX SKEENA ST,Hastings-Sunrise,497929.5,5457926
Theft from Vehicle,2022,8,18,20,46,13XX KAMLOOPS ST,Hastings-Sunrise,496017.2,5457788


TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2022,12,17,18,0,19XX WALLACE ST,West Point Grey,486137.9,5457440
Theft from Vehicle,2022,8,17,15,0,24XX CROWN ST,West Point Grey,485764.6,5456934
Theft from Vehicle,2022,10,29,20,10,25XX CAMOSUN ST,West Point Grey,485676.0,5456835
Theft from Vehicle,2022,7,18,5,30,37XX W 1ST AVE,West Point Grey,486405.9,5457641
Theft from Vehicle,2022,4,3,0,0,37XX W 2ND AVE,West Point Grey,486435.2,5457538
Theft from Vehicle,2022,8,1,0,0,38XX POINT GREY RD,West Point Grey,486325.2,5457730


TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2023,1,4,18,0,19XX SASAMAT ST,West Point Grey,484807.4,5457443
Theft from Vehicle,2023,5,10,12,0,19XX WALLACE ST,West Point Grey,486137.8,5457437
Theft from Vehicle,2023,2,20,23,0,20XX ALMA ST,West Point Grey,486484.4,5457255
Theft from Vehicle,2023,7,18,2,30,20XX HIGHBURY ST,West Point Grey,486354.0,5457305
Theft from Vehicle,2023,3,3,1,0,26XX CROWN ST,West Point Grey,485894.7,5456706
Theft from Vehicle,2023,2,3,21,0,28XX CROWN ST,West Point Grey,485890.7,5456574


<span style="font-size: 25pt;"> Data cleaning and wrangling

<span style="font-size: 15pt;"> Combining all the datasets into one

In [4]:
crimes <- rbind(hastings_2022, hastings_2023, west_point_grey_2022,west_point_grey_2023)
head(crimes)

TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
Theft from Vehicle,2022,5,16,4,20,11XX NOOTKA ST,Hastings-Sunrise,496992.1,5457928
Theft from Vehicle,2022,10,9,2,0,11XX ROSSLAND ST,Hastings-Sunrise,496691.3,5457991
Theft from Vehicle,2022,4,28,17,0,12XX LILLOOET ST,Hastings-Sunrise,497174.1,5457852
Theft from Vehicle,2022,4,17,5,17,12XX ROSSLAND ST,Hastings-Sunrise,496689.9,5457876
Theft from Vehicle,2022,11,8,22,0,12XX SKEENA ST,Hastings-Sunrise,497929.5,5457926
Theft from Vehicle,2022,8,18,20,46,13XX KAMLOOPS ST,Hastings-Sunrise,496017.2,5457788


<span style="font-size: 15pt;">  Cleaning the dataset

In [ ]:
crimes_cleaned <-  crimes |> #removing unwanted columns from the dataset
select(-X & -Y)

head(crimes_cleaned)

TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Theft from Vehicle,2022,5,16,4,20,11XX NOOTKA ST,Hastings-Sunrise
Theft from Vehicle,2022,10,9,2,0,11XX ROSSLAND ST,Hastings-Sunrise
Theft from Vehicle,2022,4,28,17,0,12XX LILLOOET ST,Hastings-Sunrise
Theft from Vehicle,2022,4,17,5,17,12XX ROSSLAND ST,Hastings-Sunrise
Theft from Vehicle,2022,11,8,22,0,12XX SKEENA ST,Hastings-Sunrise
Theft from Vehicle,2022,8,18,20,46,13XX KAMLOOPS ST,Hastings-Sunrise


<span style="font-size: 15pt;"> Separating Violent crimes of two different neighbourhood

In [9]:
violent_crimes <- crimes_cleaned |> 
filter(TYPE == "Offence Against a Person" | TYPE == "Homicide") #filtering Offence Against a Person & Homicide crimes as they are considered violent


vcrimes_in_point_grey <- violent_crimes |> #filtering West Point Grey neighbourhood
filter(NEIGHBOURHOOD == "West Point Grey")

vcrimes_in_hastings <- violent_crimes |> #filtering Hastings Sunrise neighbourhood
filter(NEIGHBOURHOOD == "Hastings-Sunrise")

vcrimes_in_point_grey_count <- vcrimes_in_point_grey |> #counting the total vioent crimes in West Point Grey
nrow()

vcrimes_in_hastings_count <- vcrimes_in_hastings |> #counting the total vioent crimes in Hastings Sunrise

vcrimes_in_point_grey_count
vcrimes_in_hastings_count

ERROR: Error in parse(text = x, srcfile = src): The pipe operator requires a function call as RHS


In [10]:
monthly_average_crimes <- violent_crimes |> #counting monthly crimes in both the neighbourhoods
group_by(MONTH) |> 
summarise(count = sum(MONTH))

monthly_average_crimes

MONTH,count
<dbl>,<dbl>
1,14
2,32
3,51
4,68
5,85
6,144
7,98
8,120
9,63


## Plots

## Compute estimates of the parameter you identified across your groups. Present this in a table. If relevant, include these estimates in your plot.

# Methods: Plan

### What do you expect to find?



### What future questions could this lead to?

### What impact could such findings have?

# References